In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import vstack
import spacy
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
nlp = spacy.load("lt_core_news_lg")
all_values = pd.read_csv('pradinisi.csv', encoding='utf-8', usecols=['text', 'class']).values
stopwordsall = pd.read_csv('stopwords.csv')
stopwords = stopwordsall['stop_words'].tolist()
stop_words_set = set(stopwords)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import re
import pickle
import snowballstemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
stemmer = snowballstemmer.stemmer('lithuanian')
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd
all_values = pd.read_csv('pradinisi.csv', usecols=['text', 'class'])
filtered_valuesY = all_values.loc[all_values['class'] == 1, 'text'].values
filtered_valuesX = all_values.loc[all_values['class'] == 0, 'text'].values

Minimum number of words: 14
Maximum number of words: 320
Average number of words: 83.38461538461539


In [10]:
import re
import spacy
import snowballstemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
stemmer = snowballstemmer.stemmer('lithuanian')
nlp = spacy.load("lt_core_news_lg")
all_values = pd.read_csv('pradinisi.csv', encoding='utf-8', usecols=['text', 'class']).values
stopwordsall = pd.read_csv('stopwords.csv')
stopwords = stopwordsall['stop_words'].tolist()
stop_words_set = set(stopwords)
documents = []
documents_be = []
maxtokenX = 1
# Initialize variables
min_words = float('inf')
max_words = float('-inf')
total_words = 0

for sen in (filtered_valuesY):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(sen))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub("\d+", "", document)

    # Converting to Lowercase
    document = document.lower()
    document = document.split()
    documentbe = str(sen).split()
    document = [word for word in document if not word in stop_words_set]
    # Stemming
    document = ' '.join(document)
    document = nlp(document)
    document = [token.orth_ for token in document]
    document = stemmer.stemWords(document)
    num_words = len(document)
    min_words = min(min_words, num_words)
    max_words = max(max_words, num_words)
    total_words += num_words
    document = ' '.join(document)
    
    documents.append(document)
    # Calculate average
average_words = total_words / len(filtered_valuesX)
# Print the results
print("Minimum number of words:", min_words)
print("Maximum number of words:", max_words)
print("Average number of words:", average_words)

Minimum number of words: 1
Maximum number of words: 30
Average number of words: 8.52662721893491


11.043392504930967 12.284023668639053


In [ ]:
def sentence_corection(sentence):
    
    document = re.sub(r'\W', ' ', str(sentence))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Converting to Lowercase
    document = document.lower()
    document = re.sub("\d+", "", document)

    # Stop words filter
    document = document.split()
    document = [word for word in document if not word in stop_words_set]
    # Stemming
    document = ' '.join(document)
    document = nlp(document)
    document = [token.orth_ for token in document]
    document = stemmer.stemWords(document)
    document = ' '.join(document)
    return document

In [12]:
def get_splits(sentences):
    y = [] # sentences classes
    for sentence in sentences:
        y.append(sentence[1]) # sentences classes
    return y
y = get_splits(all_values)

In [14]:
# Vectorize the text data using CounterVectorizer (Bag of words)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=True, stratify=y,)
# Inverse
vectorizerI = TfidfVectorizer(max_features=4000)
Inverse = vectorizerI.fit_transform(documents)
X_train_I, X_test_I, y_train_I, y_test_I = train_test_split(Inverse, y, test_size=0.25, random_state=0, shuffle=True, stratify=y,)
# Apply Latent Semantic Analysis (LSA) to reduce dimensionality
from sklearn.decomposition import TruncatedSVD
# Bag of words
lsa = TruncatedSVD(n_components=450)
train_features = lsa.fit_transform(X_train)
test_features = lsa.transform(X_test)
# Inverse
lsa = TruncatedSVD(n_components=450)
train_features_I = lsa.fit_transform(X_train_I)
test_features_I = lsa.transform(X_test_I)
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
pickle.dump(vectorizerI, open('vetorizerI.pkl', 'wb'))
pickle.dump(lsa, open('lsa.pkl', 'wb'))

ValueError: Found input variables with inconsistent numbers of samples: [507, 1014]

In [ ]:
smote = SMOTE(sampling_strategy='auto')
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
vectorizerI = TfidfVectorizer(max_features=4000)
Inverse = vectorizerI.fit_transform(documents)
lsa = TruncatedSVD(n_components=600)
lsa2 = TruncatedSVD(n_components=600)
# TF
X_smote, y_smote = smote.fit_resample(X, y)
new_y = y +y_smote
S_TF = vstack([X, X_smote])
X_train, X_test, y_train, y_test = train_test_split(S_TF, new_y, test_size=0.25, random_state=0, shuffle=True, stratify=new_y)
# TF-IDF
X_smote_I, y_smote_I = smote.fit_resample(Inverse, y)
S_TF_I = vstack([Inverse, X_smote_I])
smote_inv_y = y + y_smote_I
X_train_I, X_test_I, y_train_I, y_test_I = train_test_split(S_TF_I, smote_inv_y, test_size=0.25, random_state=0, shuffle=True, stratify=smote_inv_y,)
train_features = lsa.fit_transform(X_train)
test_features = lsa.transform(X_test)
train_features_I = lsa2.fit_transform(X_train_I)
test_features_I = lsa2.transform(X_test_I)


In [ ]:
# Smote before vectors
smote = SMOTE(sampling_strategy='all', k_neighbors=450)
X_smote, y_smote = smote.fit_resample(X, y)
smote2 = SMOTE(sampling_strategy='all', k_neighbors=450)
X_smote2, y_smote2 = smote.fit_resample(Inverse, y)
new_X = vectorizer.inverse_transform(X_smote)
new_X2 = vectorizer.inverse_transform(X_smote2)
print(f"Originalus sakinys: {documents[100]}")
print(f"SMOTE ir TF: {' '.join(new_X[100])}")
print(f"Originalus sakinys: {documents[100]}")
print(f"SMOTE ir TF-IDF: {' '.join(new_X2[100])}")

In [ ]:
# Train classifiers on the reduced feature matrix
# SVM with LSA
svmLSA = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
svmLSA.fit(train_features, y_train)

# SVM with LSA and Inv
svmLSA_Inverse = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
svmLSA_Inverse.fit(train_features_I, y_train_I)

# SVM withoutLSA
svm = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
svm.fit(X_train, y_train)

# SVM withoutLSA with INV
svm_Inverse = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
svm_Inverse.fit(X_train_I, y_train_I)

# Predict labels for test data with LSA
predicted_labels_SVM_LSA = svmLSA.predict(test_features)

# Predict labels for test data
predicted_labels_SVM = svm.predict(X_test)

# Predict labels for test data with LSA and Inverse
predicted_labels_SVM_LSA_I = svmLSA_Inverse.predict(test_features_I)

# Predict labels for test data
predicted_labels_SVM_I = svm_Inverse.predict(X_test_I)
print(predicted_labels_SVM_I)
print(y_test_I)
# print(vectorizerI.inverse_transform(X_train_I[12]))
# Evaluate  SVM classifier with LSA
SVM_accuracy_LSA = accuracy_score(y_test, predicted_labels_SVM_LSA)
SVM_report_LSA = classification_report(y_test, predicted_labels_SVM_LSA)
SVM_matrix_LSA = confusion_matrix(y_test, predicted_labels_SVM_LSA)
# Evaluate  SVM classifier
SVM_accuracy = accuracy_score(y_test, predicted_labels_SVM)
SVM_report = classification_report(y_test, predicted_labels_SVM)
SVM_matrix = confusion_matrix(y_test, predicted_labels_SVM)
# Evaluate  SVM classifier with Inverse and LSA
SVM_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_SVM_LSA_I)
SVM_report_LSA_I = classification_report(y_test_I, predicted_labels_SVM_LSA_I)
SVM_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_SVM_LSA_I)
# Evaluate  SVM classifier with Inverse
SVM_accuracy_I = accuracy_score(y_test_I, predicted_labels_SVM_I)
SVM_report_I = classification_report(y_test_I, predicted_labels_SVM_I)
SVM_matrix_I = confusion_matrix(y_test_I, predicted_labels_SVM_I)
# Plot conf matrixes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
ax = ax.flatten()
svmMat = sns.heatmap(SVM_matrix, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[0])
svmMatLSA = sns.heatmap(SVM_matrix_LSA, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[1])
svmMat_I = sns.heatmap(SVM_matrix_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[2])
svmMatLSA_I = sns.heatmap(SVM_matrix_LSA_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[3])
ax[0].set_title('SVM TF')
ax[0].set_xlabel('Spėjimas', fontsize=10)
ax[0].set_ylabel('Tikra klasė', fontsize=10)
ax[1].set_title('SVM TF ir LSA')
ax[1].set_xlabel('Spėjimas', fontsize=10)
ax[1].set_ylabel('Tikra klasė', fontsize=10)
ax[2].set_title('SVM TF-IDF')
ax[2].set_xlabel('Spėjimas', fontsize=10)
ax[2].set_ylabel('Tikra klasė', fontsize=10)
ax[3].set_title('SVM TF-IDF ir LSA')
ax[3].set_xlabel('Spėjimas', fontsize=10)
ax[3].set_ylabel('Tikra klasė', fontsize=10)
# Adjust spacing between subplots and between subplots and figure edges
# Add a title to the figure
plt.subplots_adjust(wspace=0.25, hspace=0.8, top=0.92, bottom=0.08, left=0.08, right=0.92)
plt.tight_layout()
plt.show()
# Evaluate  SVM classifier
print(f"SVM \n Accuracy: {SVM_accuracy:.2f}", "\n" ,"Report: ",  "\n",SVM_report)
print(f"SVM LSA \n Accuracy: {SVM_accuracy_LSA:.2f}", "\n" ,"Report: ",  "\n",SVM_report_LSA)
print(f"SVM Inverse \n Accuracy: {SVM_accuracy_I:.2f}", "\n" ,"Report: ",  "\n",SVM_report_I)
print(f"SVM Inverse LSA \n Accuracy: {SVM_accuracy_LSA_I:.2f}", "\n" ,"Report: ",  "\n",SVM_report_LSA_I)

In [ ]:
# SVM dimensions
# Bag of words
# Define the range of number of components
n_components_range = range(50, 1001, 50)
accuracies = []
reports = []
matrices = []
accuraciesI = []
reportsI = []
matricesI = []
# Initialize lists to store evaluation results
for n_components in n_components_range:
    lsa = TruncatedSVD(n_components=n_components)
    train_features = lsa.fit_transform(X_train)
    test_features = lsa.transform(X_test)
    lsa = TruncatedSVD(n_components=n_components)
    train_features_I = lsa.fit_transform(X_train_I)
    test_features_I = lsa.transform(X_test_I)
    svmLSA = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
    svmLSA.fit(train_features, y_train)
    svmLSA_Inverse = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
    svmLSA_Inverse.fit(train_features_I, y_train_I)
    predicted_labels_SVM_LSA = svmLSA.predict(test_features)
    predicted_labels_SVM_LSA_I = svmLSA_Inverse.predict(test_features_I)
    SVM_accuracy_LSA = accuracy_score(y_test, predicted_labels_SVM_LSA)
    SVM_report_LSA = classification_report(y_test, predicted_labels_SVM_LSA)
    SVM_matrix_LSA = confusion_matrix(y_test, predicted_labels_SVM_LSA)
    SVM_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_SVM_LSA_I)
    SVM_report_LSA_I = classification_report(y_test_I, predicted_labels_SVM_LSA_I)
    SVM_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_SVM_LSA_I)
    accuracies.append(SVM_accuracy_LSA)
    reports.append(SVM_report_LSA)
    matrices.append(SVM_matrix_LSA)
    accuraciesI.append(SVM_accuracy_LSA_I)
    reportsI.append(SVM_report_LSA_I)
    matricesI.append(SVM_matrix_LSA_I)
for accuracy in accuracies:
    print(accuracy)
for accuracy in accuraciesI:
    print(accuracy)
# Print all the evaluation results
# for n_components, SVM_accuracy_LSA, SVM_report_LSA, SVM_matrix_LSA,SVM_accuracy_LSA_I,SVM_report_LSA_I,SVM_matrix_LSA_I in zip(n_components_range, accuracies, reports, matrices,accuraciesI, reportsI, matricesI):
#     print(f"n_components = {n_components}")
#     print(f"Accuracy: {SVM_accuracy_LSA}")
#     print(f"Classification Report:\n{SVM_report_LSA}")
#     print(f"Confusion Matrix:\n{SVM_matrix_LSA}\n")
#     print(f"Accuracy I : {SVM_accuracy_LSA_I}")
#     print(f"Classification Report I :\n{SVM_report_LSA_I}")
#     print(f"Confusion Matrix I :\n{SVM_matrix_LSA_I}\n")

In [ ]:
# Random Forest
random = RandomForestClassifier(n_estimators=100, max_depth=10)
random.fit(X_train, y_train)

# Random Forest with LSA
randomLSA = RandomForestClassifier(n_estimators=100, max_depth=10)
randomLSA.fit(train_features, y_train)

# Random Forest Inverse
random_I = RandomForestClassifier(n_estimators=100, max_depth=10)
random_I.fit(X_train_I, y_train_I)

# Random Forest with LSA and INV
randomLSA_I = RandomForestClassifier(n_estimators=100, max_depth=10)
randomLSA_I.fit(train_features_I, y_train_I)

# Predict labels for test data with LSA
predicted_labels_Random_LSA = randomLSA.predict(test_features)

# Predict labels for test data
predicted_labels_Random = random.predict(X_test)

# Predict labels for test data with LSA and Inverse
predicted_labels_Random_LSA_I = randomLSA_I.predict(test_features_I)

# Predict labels for test data
predicted_labels_Random_I = random_I.predict(X_test_I)

# Evaluate  Random classifier with LSA
Random_accuracy_LSA = accuracy_score(y_test, predicted_labels_Random_LSA)
Random_report_LSA = classification_report(y_test, predicted_labels_Random_LSA)
Random_matrix_LSA = confusion_matrix(y_test, predicted_labels_Random_LSA)
# Evaluate  Random classifier
Random_accuracy = accuracy_score(y_test, predicted_labels_Random)
Random_report = classification_report(y_test, predicted_labels_Random)
Random_matrix = confusion_matrix(y_test, predicted_labels_Random)
# Evaluate  Random classifier with Inverse and LSA
Random_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_Random_LSA_I)
Random_report_LSA_I = classification_report(y_test_I, predicted_labels_Random_LSA_I)
Random_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_Random_LSA_I)
# Evaluate  Randonm classifier with Inverse
Random_accuracy_I = accuracy_score(y_test_I, predicted_labels_Random_I)
Random_report_I = classification_report(y_test_I, predicted_labels_Random_I)
Random_matrix_I = confusion_matrix(y_test_I, predicted_labels_Random_I)
# Plot conf matrixes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
ax = ax.flatten()
RandomMat = sns.heatmap(Random_matrix, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[0])
RandomMatLSA = sns.heatmap(Random_matrix_LSA, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[1])
RandomMat_I = sns.heatmap(Random_matrix_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[2])
RandomMatLSA_I = sns.heatmap(Random_matrix_LSA_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[3])
ax[0].set_title('Random Forest TF')
ax[0].set_xlabel('Spėjimas', fontsize=10)
ax[0].set_ylabel('Tikra klasė', fontsize=10)
ax[1].set_title('Random Forest TF ir LSA')
ax[1].set_xlabel('Spėjimas', fontsize=10)
ax[1].set_ylabel('Tikra klasė', fontsize=10)
ax[2].set_title('Random Forest TF-IDF')
ax[2].set_xlabel('Spėjimas', fontsize=10)
ax[2].set_ylabel('Tikra klasė', fontsize=10)
ax[3].set_title('Random Forest TF-IDF ir LSA')
ax[3].set_xlabel('Spėjimas', fontsize=10)
ax[3].set_ylabel('Tikra klasė', fontsize=10)
# Adjust spacing between subplots and between subplots and figure edges
# Add a title to the figure
plt.subplots_adjust(wspace=0.25, hspace=0.7, top=0.92, bottom=0.08, left=0.08, right=0.92)
plt.tight_layout()
plt.show()
# Print Statistics
print(f"Random \n Accuracy: {Random_accuracy:.2f}", "\n" ,"Report: ",  "\n",Random_report)
print(f"Random LSA \n Accuracy: {Random_accuracy_LSA:.2f}", "\n" ,"Report: ",  "\n",Random_report_LSA)
print(f"Random Inverse \n Accuracy: {Random_accuracy_I:.2f}", "\n" ,"Report: ",  "\n",Random_report_I)
print(f"Random Inverse LSA \n Accuracy: {Random_accuracy_LSA_I:.2f}", "\n" ,"Report: ",  "\n",Random_report_LSA_I)

In [ ]:
n_components_range = range(50, 1001, 50)
accuracies = []
reports = []
matrices = []
accuraciesI = []
reportsI = []
matricesI = []
# Initialize lists to store evaluation results
for n_components in n_components_range:
    lsa = TruncatedSVD(n_components=n_components)
    train_features = lsa.fit_transform(X_train)
    test_features = lsa.transform(X_test)
    lsa = TruncatedSVD(n_components=n_components)
    train_features_I = lsa.fit_transform(X_train_I)
    test_features_I = lsa.transform(X_test_I)
    RandomLSA = RandomForestClassifier(n_estimators=150, max_depth=10)
    RandomLSA.fit(train_features, y_train)
    RandomLSA_Inverse = RandomForestClassifier(n_estimators=150, max_depth=10)
    RandomLSA_Inverse.fit(train_features_I, y_train_I)
    predicted_labels_Random_LSA = RandomLSA.predict(test_features)
    predicted_labels_Random_LSA_I = RandomLSA_Inverse.predict(test_features_I)
    Random_accuracy_LSA = accuracy_score(y_test, predicted_labels_Random_LSA)
    Random_report_LSA = classification_report(y_test, predicted_labels_Random_LSA)
    Random_matrix_LSA = confusion_matrix(y_test, predicted_labels_Random_LSA)
    Random_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_Random_LSA_I)
    Random_report_LSA_I = classification_report(y_test_I, predicted_labels_Random_LSA_I)
    Random_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_Random_LSA_I)
    accuracies.append(Random_accuracy_LSA)
    reports.append(Random_report_LSA)
    matrices.append(Random_matrix_LSA)
    accuraciesI.append(Random_accuracy_LSA_I)
    reportsI.append(Random_report_LSA_I)
    matricesI.append(Random_matrix_LSA_I)
for accuracy in accuracies:
    print(accuracy)
for accuracy in accuraciesI:
    print(accuracy)

In [ ]:
min_value = np.min(train_features)
train_features = train_features + abs(min_value)
min_valueT = np.min(test_features)
test_features = test_features + abs(min_valueT)
min_valueI = np.min(train_features_I)
train_features_I = train_features_I + abs(min_valueI)
min_valueTI = np.min(test_features_I)
test_features_I = test_features_I + abs(min_valueTI)
# Naive Bayess
NaiveB = MultinomialNB()
NaiveB.fit(X_train, y_train)

# NaiveB Forest with LSA
NaiveBLSA = MultinomialNB()
NaiveBLSA.fit(train_features, y_train)

# NaiveB Forest Inverse
NaiveB_I = MultinomialNB()
NaiveB_I.fit(X_train_I, y_train_I)

# NaiveB Forest with LSA and INV
NaiveBLSA_I = MultinomialNB()
NaiveBLSA_I.fit(train_features_I, y_train_I)

# Predict labels for test data with LSA
predicted_labels_NaiveB_LSA = NaiveBLSA.predict(test_features)

# Predict labels for test data
predicted_labels_NaiveB = NaiveB.predict(X_test)

# Predict labels for test data with LSA and Inverse
predicted_labels_NaiveB_LSA_I = NaiveBLSA_I.predict(test_features_I)

# Predict labels for test data
predicted_labels_NaiveB_I = NaiveB_I.predict(X_test_I)

# Evaluate  Naive B classifier with LSA
NaiveB_accuracy_LSA = accuracy_score(y_test, predicted_labels_NaiveB_LSA)
NaiveB_report_LSA = classification_report(y_test, predicted_labels_NaiveB_LSA)
NaiveB_matrix_LSA = confusion_matrix(y_test, predicted_labels_NaiveB_LSA)
# Evaluate  Naive B classifier
NaiveB_accuracy = accuracy_score(y_test, predicted_labels_NaiveB)
NaiveB_report = classification_report(y_test, predicted_labels_NaiveB)
NaiveB_matrix = confusion_matrix(y_test, predicted_labels_NaiveB)
# Evaluate  NaiveB classifier with Inverse and LSA
NaiveB_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_NaiveB_LSA_I)
NaiveB_report_LSA_I = classification_report(y_test_I, predicted_labels_NaiveB_LSA_I)
NaiveB_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_NaiveB_LSA_I)
# Evaluate  Naive B classifier with Inverse
NaiveB_accuracy_I = accuracy_score(y_test_I, predicted_labels_NaiveB_I)
NaiveB_report_I = classification_report(y_test_I, predicted_labels_NaiveB_I)
NaiveB_matrix_I = confusion_matrix(y_test_I, predicted_labels_NaiveB_I)

# Plot conf matrixes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
ax = ax.flatten()
NaiveBMat = sns.heatmap(NaiveB_matrix, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[0])
NaiveBMatLSA = sns.heatmap(NaiveB_matrix_LSA, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[1])
NaiveBMat_I = sns.heatmap(NaiveB_matrix_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[2])
NaiveBMatLSA_I = sns.heatmap(NaiveB_matrix_LSA_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[3])
ax[0].set_title('Naive Bayess TF')
ax[0].set_xlabel('Spėjimas', fontsize=10)
ax[0].set_ylabel('Tikra klasė', fontsize=10)
ax[1].set_title('Naive Bayess TF ir LSA')
ax[1].set_xlabel('Spėjimas', fontsize=10)
ax[1].set_ylabel('Tikra klasė', fontsize=10)
ax[2].set_title('Naive Bayess TF-IDF')
ax[2].set_xlabel('Spėjimas', fontsize=10)
ax[2].set_ylabel('Tikra klasė', fontsize=10)
ax[3].set_title('Naive Bayess TF-IDF ir LSA')
ax[3].set_xlabel('Spėjimas', fontsize=10)
ax[3].set_ylabel('Tikra klasė', fontsize=10)

# Adjust spacing between subplots and between subplots and figure edges
# Add a title to the figure
plt.subplots_adjust(wspace=0.25, hspace=0.8, top=0.92, bottom=0.08, left=0.08, right=0.92)
plt.tight_layout()
plt.show()
# Print Statistics
print(f"Naive Bayess \n Accuracy: {NaiveB_accuracy:.2f}", "\n" ,"Report: ",  "\n",NaiveB_report)
print(f"Naive Bayess LSA \n Accuracy: {NaiveB_accuracy_LSA:.2f}", "\n" ,"Report: ",  "\n",NaiveB_report_LSA)
print(f"Naive Bayess Inverse \n Accuracy: {NaiveB_accuracy_I:.2f}", "\n" ,"Report: ",  "\n",NaiveB_report_I)
print(f"Naive Bayess Inverse LSA \n Accuracy: {NaiveB_accuracy_LSA_I:.2f}", "\n" ,"Report: ",  "\n",NaiveB_report_LSA_I)

In [ ]:

n_components_range = range(50, 1001, 50)
accuracies = []
reports = []
matrices = []
accuraciesI = []
reportsI = []
matricesI = []
# Initialize lists to store evaluation results
for n_components in n_components_range:
    lsa = TruncatedSVD(n_components=n_components)
    train_features = lsa.fit_transform(X_train)
    test_features = lsa.transform(X_test)
    lsa = TruncatedSVD(n_components=n_components)
    train_features_I = lsa.fit_transform(X_train_I)
    test_features_I = lsa.transform(X_test_I)
    min_value = np.min(train_features)
    train_features = train_features + abs(min_value)
    min_valueT = np.min(test_features)
    test_features = test_features + abs(min_valueT)
    min_valueI = np.min(train_features_I)
    train_features_I = train_features_I + abs(min_valueI)
    min_valueTI = np.min(test_features_I)
    test_features_I = test_features_I + abs(min_valueTI)
    NaiveBLSA = MultinomialNB()
    NaiveBLSA.fit(train_features, y_train)
    NaiveBLSA_Inverse = MultinomialNB()
    NaiveBLSA_Inverse.fit(train_features_I, y_train_I)
    predicted_labels_NaiveB_LSA = NaiveBLSA.predict(test_features)
    predicted_labels_NaiveB_LSA_I = NaiveBLSA_Inverse.predict(test_features_I)
    NaiveB_accuracy_LSA = accuracy_score(y_test, predicted_labels_NaiveB_LSA)
    NaiveB_report_LSA = classification_report(y_test, predicted_labels_NaiveB_LSA)
    NaiveB_matrix_LSA = confusion_matrix(y_test, predicted_labels_NaiveB_LSA)
    NaiveB_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_NaiveB_LSA_I)
    NaiveB_report_LSA_I = classification_report(y_test_I, predicted_labels_NaiveB_LSA_I)
    NaiveB_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_NaiveB_LSA_I)
    accuracies.append(NaiveB_accuracy_LSA)
    reports.append(NaiveB_report_LSA)
    matrices.append(NaiveB_matrix_LSA)
    accuraciesI.append(NaiveB_accuracy_LSA_I)
    reportsI.append(NaiveB_report_LSA_I)
    matricesI.append(NaiveB_matrix_LSA_I)
for accuracy in accuracies:
    print(accuracy)
for accuracy in accuraciesI:
    print(accuracy)

In [ ]:
# Decision Trees
DecisionT = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
DecisionT.fit(X_train, y_train)

# DecisionT with LSA
DecisionTLSA = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
DecisionTLSA.fit(train_features, y_train)

# DecisionT Inverse
DecisionT_I = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
DecisionT_I.fit(X_train_I, y_train_I)

# DecisionT with LSA and INV
DecisionTLSA_I = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
DecisionTLSA_I.fit(train_features_I, y_train_I)

# Predict labels for test data with LSA
predicted_labels_DecisionT_LSA = DecisionTLSA.predict(test_features)

# Predict labels for test data
predicted_labels_DecisionT = DecisionT.predict(X_test)

# Predict labels for test data with LSA and Inverse
predicted_labels_DecisionT_LSA_I = DecisionTLSA_I.predict(test_features_I)

# Predict labels for test data
predicted_labels_DecisionT_I = DecisionT_I.predict(X_test_I)

# Evaluate  Naive B classifier with LSA
DecisionT_accuracy_LSA = accuracy_score(y_test, predicted_labels_DecisionT_LSA)
DecisionT_report_LSA = classification_report(y_test, predicted_labels_DecisionT_LSA)
DecisionT_matrix_LSA = confusion_matrix(y_test, predicted_labels_DecisionT_LSA)
# Evaluate  Naive B classifier
DecisionT_accuracy = accuracy_score(y_test, predicted_labels_DecisionT)
DecisionT_report = classification_report(y_test, predicted_labels_DecisionT)
DecisionT_matrix = confusion_matrix(y_test, predicted_labels_DecisionT)
# Evaluate  DecisionT classifier with Inverse and LSA
DecisionT_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_DecisionT_LSA_I)
DecisionT_report_LSA_I = classification_report(y_test_I, predicted_labels_DecisionT_LSA_I)
DecisionT_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_DecisionT_LSA_I)
# Evaluate  Naive B classifier with Inverse
DecisionT_accuracy_I = accuracy_score(y_test_I, predicted_labels_DecisionT_I)
DecisionT_report_I = classification_report(y_test_I, predicted_labels_DecisionT_I)
DecisionT_matrix_I = confusion_matrix(y_test_I, predicted_labels_DecisionT_I)

# Plot conf matrixes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
ax = ax.flatten()
DecisionTMat = sns.heatmap(DecisionT_matrix, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[0])
DecisionTMatLSA = sns.heatmap(DecisionT_matrix_LSA, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[1])
DecisionTMat_I = sns.heatmap(DecisionT_matrix_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[2])
DecisionTMatLSA_I = sns.heatmap(DecisionT_matrix_LSA_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[3])
ax[0].set_title('Decision Trees TF')
ax[0].set_xlabel('Spėjimas', fontsize=10)
ax[0].set_ylabel('Tikra klasė', fontsize=10)
ax[1].set_title('Decision Trees TF ir LSA')
ax[1].set_xlabel('Spėjimas', fontsize=10)
ax[1].set_ylabel('Tikra klasė', fontsize=10)
ax[2].set_title('Decision Trees TF-IDF')
ax[2].set_xlabel('Spėjimas', fontsize=10)
ax[2].set_ylabel('Tikra klasė', fontsize=10)
ax[3].set_title('Decision Trees TF-IDF ir LSA')
ax[3].set_xlabel('Spėjimas', fontsize=10)
ax[3].set_ylabel('Tikra klasė', fontsize=10)

# Adjust spacing between subplots and between subplots and figure edges
# Add a title to the figure
plt.subplots_adjust(wspace=0.25, hspace=0.8, top=0.92, bottom=0.08, left=0.08, right=0.92)
plt.tight_layout()
plt.show()
# Print Statistics
print(f"Decision Trees \n Accuracy: {DecisionT_accuracy:.2f}", "\n" ,"Report: ",  "\n",DecisionT_report)
print(f"Decision Trees LSA \n Accuracy: {DecisionT_accuracy_LSA:.2f}", "\n" ,"Report: ",  "\n",DecisionT_report_LSA)
print(f"Decision Trees Inverse \n Accuracy: {DecisionT_accuracy_I:.2f}", "\n" ,"Report: ",  "\n",DecisionT_report_I)
print(f"Decision Trees Inverse LSA \n Accuracy: {DecisionT_accuracy_LSA_I:.2f}", "\n" ,"Report: ",  "\n",DecisionT_report_LSA_I)

In [ ]:
n_components_range = range(50, 1001, 50)
accuracies = []
reports = []
matrices = []
accuraciesI = []
reportsI = []
matricesI = []
# Initialize lists to store evaluation results
for n_components in n_components_range:
    lsa = TruncatedSVD(n_components=n_components)
    train_features = lsa.fit_transform(X_train)
    test_features = lsa.transform(X_test)
    lsa = TruncatedSVD(n_components=n_components)
    train_features_I = lsa.fit_transform(X_train_I)
    test_features_I = lsa.transform(X_test_I)
    DecisionTLSA = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
    DecisionTLSA.fit(train_features, y_train)
    DecisionTLSA_Inverse = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
    DecisionTLSA_Inverse.fit(train_features_I, y_train_I)
    predicted_labels_DecisionT_LSA = DecisionTLSA.predict(test_features)
    predicted_labels_DecisionT_LSA_I = DecisionTLSA_Inverse.predict(test_features_I)
    DecisionT_accuracy_LSA = accuracy_score(y_test, predicted_labels_DecisionT_LSA)
    DecisionT_report_LSA = classification_report(y_test, predicted_labels_DecisionT_LSA)
    DecisionT_matrix_LSA = confusion_matrix(y_test, predicted_labels_DecisionT_LSA)
    DecisionT_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_DecisionT_LSA_I)
    DecisionT_report_LSA_I = classification_report(y_test_I, predicted_labels_DecisionT_LSA_I)
    DecisionT_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_DecisionT_LSA_I)
    accuracies.append(DecisionT_accuracy_LSA)
    reports.append(DecisionT_report_LSA)
    matrices.append(DecisionT_matrix_LSA)
    accuraciesI.append(DecisionT_accuracy_LSA_I)
    reportsI.append(DecisionT_report_LSA_I)
    matricesI.append(DecisionT_matrix_LSA_I)
for accuracy in accuracies:
    print(accuracy)
for accuracy in accuraciesI:
    print(accuracy)

In [ ]:
# Perceptron
MPLPercep = MLPClassifier(hidden_layer_sizes=(350,500))
MPLPercep.fit(X_train, y_train)

# MPLPercep with LSA
MPLPercepLSA = MLPClassifier(hidden_layer_sizes=(350,500))
MPLPercepLSA.fit(train_features, y_train)

# MPLPercep Inverse
MPLPercep_I = MLPClassifier(hidden_layer_sizes=(350,500))
MPLPercep_I.fit(X_train_I, y_train_I)

# MPLPercep with LSA and INV
MPLPercepLSA_I = MLPClassifier(hidden_layer_sizes=(350,500))
MPLPercepLSA_I.fit(train_features_I, y_train_I)

# Predict labels for test data with LSA
predicted_labels_MPLPercep_LSA = MPLPercepLSA.predict(test_features)

# Predict labels for test data
predicted_labels_MPLPercep = MPLPercep.predict(X_test)

# Predict labels for test data with LSA and Inverse
predicted_labels_MPLPercep_LSA_I = MPLPercepLSA_I.predict(test_features_I)

# Predict labels for test data
predicted_labels_MPLPercep_I = MPLPercep_I.predict(X_test_I)

# Evaluate  Naive B classifier with LSA
MPLPercep_accuracy_LSA = accuracy_score(y_test, predicted_labels_MPLPercep_LSA)
MPLPercep_report_LSA = classification_report(y_test, predicted_labels_MPLPercep_LSA)
MPLPercep_matrix_LSA = confusion_matrix(y_test, predicted_labels_MPLPercep_LSA)
# Evaluate  Naive B classifier
MPLPercep_accuracy = accuracy_score(y_test, predicted_labels_MPLPercep)
MPLPercep_report = classification_report(y_test, predicted_labels_MPLPercep)
MPLPercep_matrix = confusion_matrix(y_test, predicted_labels_MPLPercep)
# Evaluate  MPLPercep classifier with Inverse and LSA
MPLPercep_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_MPLPercep_LSA_I)
MPLPercep_report_LSA_I = classification_report(y_test_I, predicted_labels_MPLPercep_LSA_I)
MPLPercep_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_MPLPercep_LSA_I)
# Evaluate  Naive B classifier with Inverse
MPLPercep_accuracy_I = accuracy_score(y_test_I, predicted_labels_MPLPercep_I)
MPLPercep_report_I = classification_report(y_test_I, predicted_labels_MPLPercep_I)
MPLPercep_matrix_I = confusion_matrix(y_test_I, predicted_labels_MPLPercep_I)

# Plot conf matrixes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
ax = ax.flatten()
MPLPercepMat = sns.heatmap(MPLPercep_matrix, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[0])
MPLPercepMatLSA = sns.heatmap(MPLPercep_matrix_LSA, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[1])
MPLPercepMat_I = sns.heatmap(MPLPercep_matrix_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[2])
MPLPercepMatLSA_I = sns.heatmap(MPLPercep_matrix_LSA_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[3])
ax[0].set_title('MLP TF')
ax[0].set_xlabel('Spėjimas', fontsize=10)
ax[0].set_ylabel('Tikra klasė', fontsize=10)
ax[1].set_title('MLP TF ir LSA')
ax[1].set_xlabel('Spėjimas', fontsize=10)
ax[1].set_ylabel('Tikra klasė', fontsize=10)
ax[2].set_title('MLP TF-IDF')
ax[2].set_xlabel('Spėjimas', fontsize=10)
ax[2].set_ylabel('Tikra klasė', fontsize=10)
ax[3].set_title('MLP TF-IDF ir LSA')
ax[3].set_xlabel('Spėjimas', fontsize=10)
ax[3].set_ylabel('Tikra klasė', fontsize=10)

# Adjust spacing between subplots and between subplots and figure edges
# Add a title to the figure
plt.subplots_adjust(wspace=0.25, hspace=0.8, top=0.92, bottom=0.08, left=0.08, right=0.92)
plt.tight_layout()
plt.show()
# Print Statistics
print(f"MLP Perceptron \n Accuracy: {MPLPercep_accuracy:.2f}", "\n" ,"Report: ",  "\n",MPLPercep_report)
print(f"MLP Perceptron LSA \n Accuracy: {MPLPercep_accuracy_LSA:.2f}", "\n" ,"Report: ",  "\n",MPLPercep_report_LSA)
print(f"MLP Perceptron Inverse \n Accuracy: {MPLPercep_accuracy_I:.2f}", "\n" ,"Report: ",  "\n",MPLPercep_report_I)
print(f"MLP Perceptron Inverse LSA \n Accuracy: {MPLPercep_accuracy_LSA_I:.2f}", "\n" ,"Report: ",  "\n",MPLPercep_report_LSA_I)

In [ ]:
n_components_range = range(50, 1001, 50)
accuracies = []
reports = []
matrices = []
accuraciesI = []
reportsI = []
matricesI = []
# Initialize lists to store evaluation results
for n_components in n_components_range:
    lsa = TruncatedSVD(n_components=n_components)
    train_features = lsa.fit_transform(X_train)
    test_features = lsa.transform(X_test)
    lsa = TruncatedSVD(n_components=n_components)
    train_features_I = lsa.fit_transform(X_train_I)
    test_features_I = lsa.transform(X_test_I)
    PerceptLSA = MLPClassifier(hidden_layer_sizes=(350,500))
    PerceptLSA.fit(train_features, y_train)
    PerceptLSA_Inverse = MLPClassifier(hidden_layer_sizes=(350,500))
    PerceptLSA_Inverse.fit(train_features_I, y_train_I)
    predicted_labels_Percept_LSA = PerceptLSA.predict(test_features)
    predicted_labels_Percept_LSA_I = PerceptLSA_Inverse.predict(test_features_I)
    Percept_accuracy_LSA = accuracy_score(y_test, predicted_labels_Percept_LSA)
    Percept_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_Percept_LSA_I)
    accuracies.append(Percept_accuracy_LSA)
    accuraciesI.append(Percept_accuracy_LSA_I)
    print(f"n_components = {n_components}, accuracy = {Percept_accuracy_LSA}, accuracyI = {Percept_accuracy_LSA_I}")

In [ ]:
for accuracy in accuracies:
    print(accuracy)
for accuracy in accuraciesI:
    print(accuracy)

In [ ]:
# Gradient Boosting
GradBoost = GradientBoostingClassifier(max_depth=10, n_estimators=100)
GradBoost.fit(X_train, y_train)

# GradBoost with LSA
GradBoostLSA = GradientBoostingClassifier(max_depth=10, n_estimators=100)
GradBoostLSA.fit(train_features, y_train)

# GradBoost Inverse
GradBoost_I = GradientBoostingClassifier(max_depth=10, n_estimators=100)
GradBoost_I.fit(X_train_I, y_train_I)

# GradBoost with LSA and INV
GradBoostLSA_I = GradientBoostingClassifier(max_depth=10, n_estimators=100)
GradBoostLSA_I.fit(train_features_I, y_train_I)

# Predict labels for test data with LSA
predicted_labels_GradBoost_LSA = GradBoostLSA.predict(test_features)

# Predict labels for test data
predicted_labels_GradBoost = GradBoost.predict(X_test)

# Predict labels for test data with LSA and Inverse
predicted_labels_GradBoost_LSA_I = GradBoostLSA_I.predict(test_features_I)

# Predict labels for test data
predicted_labels_GradBoost_I = GradBoost_I.predict(X_test_I)

# Evaluate  Naive B classifier with LSA
GradBoost_accuracy_LSA = accuracy_score(y_test, predicted_labels_GradBoost_LSA)
GradBoost_report_LSA = classification_report(y_test, predicted_labels_GradBoost_LSA)
GradBoost_matrix_LSA = confusion_matrix(y_test, predicted_labels_GradBoost_LSA)
# Evaluate  Naive B classifier
GradBoost_accuracy = accuracy_score(y_test, predicted_labels_GradBoost)
GradBoost_report = classification_report(y_test, predicted_labels_GradBoost)
GradBoost_matrix = confusion_matrix(y_test, predicted_labels_GradBoost)
# Evaluate  GradBoost classifier with Inverse and LSA
GradBoost_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_GradBoost_LSA_I)
GradBoost_report_LSA_I = classification_report(y_test_I, predicted_labels_GradBoost_LSA_I)
GradBoost_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_GradBoost_LSA_I)
# Evaluate  Naive B classifier with Inverse
GradBoost_accuracy_I = accuracy_score(y_test_I, predicted_labels_GradBoost_I)
GradBoost_report_I = classification_report(y_test_I, predicted_labels_GradBoost_I)
GradBoost_matrix_I = confusion_matrix(y_test_I, predicted_labels_GradBoost_I)

# Plot conf matrixes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
ax = ax.flatten()
GradBoostMat = sns.heatmap(GradBoost_matrix, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[0])
GradBoostMatLSA = sns.heatmap(GradBoost_matrix_LSA, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[1])
GradBoostMat_I = sns.heatmap(GradBoost_matrix_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[2])
GradBoostMatLSA_I = sns.heatmap(GradBoost_matrix_LSA_I, annot=True, cmap='Blues',fmt='d',linewidth=0.01,linecolor="#222", ax=ax[3])
ax[0].set_title('Gradient Boosted Trees TF')
ax[0].set_xlabel('Spėjimas', fontsize=10)
ax[0].set_ylabel('Tikra klasė', fontsize=10)
ax[1].set_title('Gradient Boosted Trees TF ir LSA')
ax[1].set_xlabel('Spėjimas', fontsize=10)
ax[1].set_ylabel('Tikra klasė', fontsize=10)
ax[2].set_title('Gradient Boosted Trees TF-IDF')
ax[2].set_xlabel('Spėjimas', fontsize=10)
ax[2].set_ylabel('Tikra klasė', fontsize=10)
ax[3].set_title('Gradient Boosted Trees TF-IDF ir LSA')
ax[3].set_xlabel('Spėjimas', fontsize=10)
ax[3].set_ylabel('Tikra klasė', fontsize=10)

# Adjust spacing between subplots and between subplots and figure edges
# Add a title to the figure
plt.subplots_adjust(wspace=0.25, hspace=0.8, top=0.92, bottom=0.08, left=0.08, right=0.92)
plt.tight_layout()
plt.show()
# Print Statistics
print(f"Gradient Boosting Trees \n Accuracy: {GradBoost_accuracy:.2f}", "\n" ,"Report: ",  "\n",GradBoost_report)
print(f"Gradient Boosting Trees LSA \n Accuracy: {GradBoost_accuracy_LSA:.2f}", "\n" ,"Report: ",  "\n",GradBoost_report_LSA)
print(f"Gradient Boosting Trees Inverse \n Accuracy: {GradBoost_accuracy_I:.2f}", "\n" ,"Report: ",  "\n",GradBoost_report_I)
print(f"Gradient Boosting Trees Inverse LSA \n Accuracy: {GradBoost_accuracy_LSA_I:.2f}", "\n" ,"Report: ",  "\n",GradBoost_report_LSA_I)

In [ ]:
n_components_range = range(50, 1001, 50)
accuracies = []
reports = []
matrices = []
accuraciesI = []
reportsI = []
matricesI = []
# Initialize lists to store evaluation results
for n_components in n_components_range:
    lsa = TruncatedSVD(n_components=n_components)
    train_features = lsa.fit_transform(X_train)
    test_features = lsa.transform(X_test)
    lsa = TruncatedSVD(n_components=n_components)
    train_features_I = lsa.fit_transform(X_train_I)
    test_features_I = lsa.transform(X_test_I)
    GradBoostLSA = GradientBoostingClassifier(max_depth=10, n_estimators=100)
    GradBoostLSA.fit(train_features, y_train)
    GradBoostLSA_Inverse = GradientBoostingClassifier(max_depth=10, n_estimators=100)
    GradBoostLSA_Inverse.fit(train_features_I, y_train_I)
    predicted_labels_GradBoost_LSA = GradBoostLSA.predict(test_features)
    predicted_labels_GradBoost_LSA_I = GradBoostLSA_Inverse.predict(test_features_I)
    GradBoost_accuracy_LSA = accuracy_score(y_test, predicted_labels_GradBoost_LSA)
    GradBoost_report_LSA = classification_report(y_test, predicted_labels_GradBoost_LSA)
    GradBoost_matrix_LSA = confusion_matrix(y_test, predicted_labels_GradBoost_LSA)
    GradBoost_accuracy_LSA_I = accuracy_score(y_test_I, predicted_labels_GradBoost_LSA_I)
    GradBoost_report_LSA_I = classification_report(y_test_I, predicted_labels_GradBoost_LSA_I)
    GradBoost_matrix_LSA_I = confusion_matrix(y_test_I, predicted_labels_GradBoost_LSA_I)
    accuracies.append(GradBoost_accuracy_LSA)
    reports.append(GradBoost_report_LSA)
    matrices.append(GradBoost_matrix_LSA)
    accuraciesI.append(GradBoost_accuracy_LSA_I)
    reportsI.append(GradBoost_report_LSA_I)
    matricesI.append(GradBoost_matrix_LSA_I)
    print(f"n_components = {n_components}, accuracy = {GradBoost_accuracy_LSA}, accuracyI = {GradBoost_accuracy_LSA_I}")

In [ ]:
for accuracy in accuracies:
    print(accuracy)
for accuracy in accuraciesI:
    print(accuracy)

In [ ]:
# Parametrai
leafs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
accuracies = []
accuraciesI = []
for leaf in leafs: 
    svm = SVC(C=leaf, kernel='linear', degree=3, gamma='auto')
    svm.fit(X_train, y_train)
    svm_I = SVC(C=leaf, kernel='linear', degree=3, gamma='auto')
    svm_I.fit(X_train_I, y_train_I)
    predicted_labels_svm = svm.predict(X_test)
    predicted_labels_svm_I = svm_I.predict(X_test_I)
    svm_accuracy = accuracy_score(y_test, predicted_labels_svm)
    svm_accuracy_I = accuracy_score(y_test_I, predicted_labels_svm_I)
    accuracies.append(svm_accuracy)
    accuraciesI.append(svm_accuracy_I)

for accuracy in accuracies:
    print(round(accuracy, 2))
print('Inverse \n')
for accuracy in accuraciesI:
    print(round(accuracy, 2))

In [ ]:
# Random Forest Inverse
random_I = RandomForestClassifier(n_estimators=100, max_depth=10)
random_I.fit(X_train_I, y_train_I)
randomF = 'randomF.pkl'
pickle.dump(random_I, open(randomF, 'wb'))

In [ ]:
# GradBoost with LSA and INV
# Inverse

GradBoostLSA_I = GradientBoostingClassifier(max_depth=10, n_estimators=100)
GradBoostLSA_I.fit(train_features_I, y_train_I)
GradBoost = 'GradBoost.pkl'
pickle.dump(GradBoostLSA_I, open(GradBoost, 'wb'))

In [ ]:
DecisionT_I = DecisionTreeClassifier(min_samples_split=2, max_depth=30)
DecisionT_I.fit(X_train_I, y_train_I)
decisonT = 'decisionT.pkl'
pickle.dump(DecisionT_I, open(decisonT, 'wb'))


In [ ]:
# SVM withoutLSA with INV
svm_Inverse = SVC(C=0.9, kernel='linear', degree=3, gamma='auto')
svm_Inverse.fit(X_train_I, y_train_I)
svmmodel = 'svm.pkl'
pickle.dump(svm_Inverse, open(svmmodel, 'wb'))

In [ ]:
NaiveB = MultinomialNB()
NaiveB.fit(X_train, y_train)
naivebmodel = 'naivebmodel.pkl'
pickle.dump(NaiveB, open(naivebmodel, 'wb'))

In [ ]:
# MPLPercep Inverse
MPLPercep_I = MLPClassifier(hidden_layer_sizes=(350,500))
MPLPercep_I.fit(X_train_I, y_train_I)
mlpmodel = 'mlpmodel.pkl'
pickle.dump(MPLPercep_I, open(mlpmodel, 'wb'))

In [ ]:
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))
vectorizerI = pickle.load(open('vetorizerI.pkl', 'rb'))
lsa = pickle.load(open('lsa.pkl', 'rb'))


In [ ]:
dt = pickle.load(open('decisionT.pkl', 'rb'))
nb = pickle.load(open('naivebmodel.pkl', 'rb'))
svm = pickle.load(open('svm.pkl', 'rb'))
rt = pickle.load(open('randomF.pkl', 'rb'))
mlp = pickle.load(open('mlpmodel.pkl', 'rb'))
gb = pickle.load(open('GradBoost.pkl', 'rb'))

In [ ]:
various_sentences = [
    "Zala patenkintas visada yra,vanagas jau skundziasi tai jau sloguoja neisimegojes visa laika jam problemos aplinkui",
    "viskas gerai nepanikuokit stebetojai ,šaunuolis visad su jumoru nenusimines",
    "Svarbiausia, kad nuotaikos ir pozityvo nenusimušo",
    "reikėtų juos apdaužyti loxus",
    "suspardysiu tave gaidį",
    "jeigu tave surasčiau, tai sudaužyčiau",
    "kategorijos, kurių skonis keičiasi priklausomai nuo to, kokį pagrindinį priedą pasirinksi",
    "dar parėkausi ir baigsis blogai tau",
    "atvažiuosiu pas tave ir primušiu",
    "15min prenumerata: https://15min.lt/prenumerata/apie",
    "loxas tu eisi, butum is mano miesto sudauzyciau tave",
    "puikus papildas turintis įvairių formų baltymų, specialiai pritaikytas aktyvioms moterims.",
    "uzmusti reikia tave",
    "Geriausi PASKUTINĖS MINUTĖS pasiūlymai Graikijoje"
]
various_sentences_labels = [0,0,0,1,1,1,0,1,1,0,1,0,1,0]
for i in range(len(various_sentences)):
    correct_sentence = sentence_corection(various_sentences[i])
    predictionOnSVM = svm.predict(vectorizerI.transform([correct_sentence]))
    predictionOnRandom = rt.predict(vectorizerI.transform([correct_sentence]))
    predictionOnDT = dt.predict(vectorizerI.transform([correct_sentence]))
    predictionOnMLP = mlp.predict(vectorizerI.transform([correct_sentence]))
    predictionOnNB = nb.predict(vectorizer.transform([correct_sentence]))
    predictionOnGT = gb.predict(lsa.transform(vectorizerI.transform([correct_sentence])))
    print(f"{i+1} SVM :  {predictionOnSVM[0]} | RF:  {predictionOnRandom[0]} | NB:  {predictionOnNB[0]} DT: {predictionOnDT[0]} | GB: {predictionOnGT[0]} | MLP: {predictionOnMLP[0]}  Real Class: {various_sentences_labels[i]}\n")
for i in range(len(various_sentences)):
    print(f"{i+1}. {various_sentences[i]}")